In [1]:
import torch , gc
import torch.nn as nn
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel,AutoTokenizer, AutoModel
from tqdm import tqdm
import tensorflow as tf 
import os
import json
import pandas as pd

c:\Users\satya\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650 Ti'

In [3]:
import pandas as pd

data = pd.read_table('fra.txt', header= None)
data.rename(columns= {0: 'English', 1: 'French', 2: 'Citation'}, inplace= True)
data = data[:10000]
data

,English,French,Citation
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
9995,We've made it.,Nous avons réussi.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
9996,We've made it.,Nous y sommes parvenus.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
9997,We've matured.,Nous avons mûri.,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
9998,We've no time.,Nous n'avons pas le temps.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [4]:
pattern = r"[!'#$%&()*+,-./:;<=>?@[\]^`{|}~“”‘’«»‹›„‚–—…·•¡¿’\"\']"

eng_sent, french_sent = [], []

for e in range(len(data['English'])):
    eng_sent.append(re.sub(pattern, "", data['English'][e]))
    french_sent.append(re.sub(pattern, "", data['French'][e]))
#eng_sent[229801]


In [5]:
print(len(eng_sent))
print(len(french_sent))

10000
10000


In [6]:

bert_tiny_tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
Bert_tiny_model = AutoModel.from_pretrained("prajjwal1/bert-tiny")

In [7]:
Bert_tiny_model.device

device(type='cpu')

In [10]:
def text_embedding2(batch_tokens, max_len, model, tokenizer):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Reduce the batch size if the input is too large for GPU memory
    batch_size = len(batch_tokens)
    max_batch_size = 32  # You can adjust this value based on your GPU memory capacity

    while batch_size > max_batch_size:
        batch_tokens = batch_tokens[:batch_size // 2]  # Halve the batch size
        batch_size = len(batch_tokens)

    batch_padded_tokens = [tokens + [tokenizer.pad_token_id] * (max_len - len(tokens))
                           for tokens in batch_tokens]

    tokens_tensor = torch.tensor(batch_padded_tokens).to(device=device)
    with torch.no_grad():
        output = model(tokens_tensor)
        embeddings = output.last_hidden_state
        torch.cuda.empty_cache()

    return embeddings

In [11]:

def get_embeddings(max_length,batch_size,tokens,model,tokenizer):

    embedding_trans = []
    for i in tqdm(range(0, len(tokens), batch_size), "Embedding", colour= "green"):
        batch_token = tokens[i : i+batch_size]
        embedding_trans.extend(text_embedding2(batch_token,max_length,model,tokenizer))

    return embedding_trans

In [12]:
torch.cuda.empty_cache()
english_tokens = [bert_tiny_tokenizer.encode(text,add_special_tokens = True,padding='max_length',max_length=104) for text in eng_sent]
french_token = [bert_tiny_tokenizer.encode(text,add_special_tokens = True,padding='max_length',max_length=104) for text in french_sent]
English_embeddings = get_embeddings(max_length=104,batch_size=32,tokens=english_tokens,model=Bert_tiny_model,tokenizer=bert_tiny_tokenizer)
print('----------------------English embededding done -------------------')
English_embeddings = torch.stack(English_embeddings).to(device='cpu')
torch.cuda.empty_cache()
French_embeddings = get_embeddings(max_length=104,batch_size=32,tokens=french_token,model=Bert_tiny_model,tokenizer=bert_tiny_tokenizer)
print('----------------------French embededding done -------------------')
French_embeddings = torch.stack(French_embeddings).to(device='cpu')
torch.cuda.empty_cache()

Embedding: 100%|██████████| 1563/1563 [00:14<00:00, 110.55it/s]


----------------------English embededding done -------------------


Embedding: 100%|██████████| 1563/1563 [00:12<00:00, 125.68it/s]


----------------------French embededding done -------------------


In [14]:
English_embeddings.size()

torch.Size([50000, 104, 128])

In [15]:
import numpy as np

def positional_encoding(embedding_vectors,batch_size):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    records,max_length, d_model = embedding_vectors.size()
    
    even_i = torch.arange(0 , d_model , 2).float()
    even_denominator = torch.pow(10000, even_i/d_model)
    odd_i = torch.arange(1 , d_model , 2).float()
    odd_denominator = torch.pow(10000, (odd_i -1)/d_model)

    positions = torch.arange(max_length,dtype=torch.float).reshape(max_length,1)

    even_pe = torch.sin(positions/even_denominator)
    odd_pe = torch.sin(positions/even_denominator)
    stacked = torch.stack([even_pe , odd_pe] , dim  = 2)
    PE = torch.flatten(stacked,start_dim=1,end_dim=2)
    PE = torch.tile(PE,(batch_size,1,1))
    test_list=[]

    for i in tqdm(range(0 ,records,batch_size), "Positional_Encoding", colour= "green"):
        batch = embedding_vectors[i:i+batch_size]
        test_list.append(batch + PE)
    test_list = torch.stack(test_list).to(device=device)
    test_list= torch.flatten(test_list,start_dim=0,end_dim=1).to(device=device)
    return test_list

# Example usage:



In [16]:
English_position_encoded= positional_encoding(English_embeddings,batch_size=50)
English_position_encoded= English_position_encoded.to(device='cpu')
torch.cuda.empty_cache()
French_position_encoded = positional_encoding(French_embeddings,batch_size=50)
French_position_encoded= French_position_encoded.to(device='cpu')
torch.cuda.empty_cache()

Positional_Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

Positional_Encoding: 100%|██████████| 1000/1000 [00:00<00:00, 1571.59it/s]


In [2]:
import torch
torch.cuda.empty_cache()

In [6]:
from transformer import Transformer # This is from transformer.py

In [7]:

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb=32'
d_model = 128
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 104
ffn_hidden = 2048
num_layers_encoder = 1
num_layers_decoder = 1
transformer = Transformer(d_model = 128,
                    ffn_hidden = 256,
                    num_heads = 8,
                    drop_prob = 0.1,
                    max_sequence_length = 104,
                    num_layers_encoder = 2,
                    num_layers_decoder = 1,
                    batch_size=50
                    )

mask = torch.full([max_sequence_length, max_sequence_length] , float('-inf'))
mask = torch.triu(mask, diagonal=1) # Mask initialization for masked attention
decoder_output = transformer(eng_sent[:1000],french_sent[:1000],mask)




ENCODER ACTIVATED
tokenization done 



Positional_Encoding: 100%|██████████| 20/20 [00:00<00:00, 1179.87it/s]


POSITIONAL ENCODING IS DONE 
 torch.Size([1000, 104, 128])
SENTENCE EMBEDDING DONE 
 torch.Size([1000, 104, 128])
MEMORY USAGE AFTER ENCODING 
 |===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  26754 KiB |  78754 KiB |   3371 MiB |   3345 MiB |
|       from large pool |  23581 KiB |  75581 KiB |   3359 MiB |   3336 MiB |
|       from small pool |   3173 KiB |   3348 KiB |     11 MiB |      8 MiB |
|---------------------------------------------------------------------------

Positional_Encoding: 100%|██████████| 20/20 [00:00<00:00, 1180.26it/s]


POSITIONAL ENCODING IS DONE 
 torch.Size([1000, 104, 128])
SENTENCE EMBEDDING DONE 
 torch.Size([1000, 104, 128])
------- DEOCDER LAYER NUMBER 1----------- 

MASKED SELF ATTENTION
qkv in cpu
qkv in cpu
qkv in cpu
scaled.size() : torch.Size([1000, 8, 104, 104])
values in  cuda:0
Multi headed hattention done , size=: torch.Size([1000, 104, 128]) 

DROP OUT 1
ADD + LAYER NORMALIZATION 1
Layer normalization done , Size=: torch.Size([1000, 104, 128]) 

CROSS ATTENTION
scaled.size() : torch.Size([1000, 8, 104, 104])
values in  cuda:0
Cross attention completed size=: torch.Size([1000, 104, 128]) 

DROP OUT 2 

ADD + LAYER NORMALIZATION 2
Layer normalization done , Size=: torch.Size([1000, 104, 128]) 

FEED FORWARD 1
x feed forward network: torch.Size([1000, 104, 128])
DROP OUT 3 

ADD + LAYER NORMALIZATION 3
Layer normalization done , Size=: torch.Size([1000, 104, 128]) 

DECODER COMPLETED
MEMORY AFTER DECODER ACTION 
 |=========================================================================

In [10]:
import torch
print(torch.cuda.memory_summary(abbreviated=True))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   3391 MiB |   3442 MiB |  16292 MiB |  12901 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   3391 MiB |   3442 MiB |  16292 MiB |  12901 MiB |
|---------------------------------------------------------------------------|
| Requested memory      |   3387 MiB |   3438 MiB |  16151 MiB |  12763 MiB |
|---------------------------------------------------------------

In [13]:
path = "encoded_files"
if not os.path.exists(path):
   os.makedirs(path)
   print("The new directory is created!")
json_path = os.path.join(os.getcwd(),path)
json_path

'd:\\projects\\Machine-Translation\\embedding_files'

# writing the english embeddings in a json file and uploading it for time saving

In [17]:
eng_embedding_dict = {}
eng_embeddings_parquet_path = 'eng_embeds.parquet'
for i in range(len(eng_embeddings)):
    eng_embedding_dict[i] = eng_embeddings[i].tolist()
#eng_embedding_dict[1]
#eng_embedding_dict_json_object = json.dumps(eng_embedding_dict, indent = 4)
eng_embedding_df = pd.DataFrame(eng_embedding_dict)
eng_embedding_df.to_parquet(os.path.join(json_path,eng_embeddings_parquet_path))
#with open(os.path.join(json_path,eng_embeddings_json_path), "w") as outfile:
    #outfile.write(eng_embedding_dict_json_object)

# writing the french embeddings for time saving

In [18]:
french_embedding_dict = {}
french_embeddings_parquet_path = 'french_embeds.parquet'
for i in range(len(eng_embeddings)):
    french_embedding_dict[i] = fr_embeddings[i].tolist()
#eng_embedding_dict[1]
#french_embedding_dict_json_object = json.dumps(french_embedding_dict, indent = 4) 
fr_embedding_df = pd.DataFrame(french_embedding_dict)
fr_embedding_df.to_parquet(os.path.join(json_path,french_embeddings_parquet_path))
#with open(os.path.join(json_path,french_embeddings_json_path), "w") as outfile:
    #outfile.write(french_embedding_dict_json_object)